# Introduction to Digital Earth Australia

**Notebook currently compatible with the `NCI`|`DEA Sandbox` environment only**

### General advice (delete this cell before submitting for review)

- When choosing a location for your analysis, **select an area that has data on both the `NCI` and `DEA Sandbox`** to allow your code to be run on both environments. 
For example, you can check this for Landsat using the [DEA Explorer](https://explorer.sandbox.dea.ga.gov.au/ga_ls5t_ard_3/1990) (use the drop-down menu to view all products). 
As of September 2019, the `DEA Sandbox` has a single year of continental Landsat data for 2015-16, and the full 1987-onward time-series for three locations (Perth WA, Brisbane QLD, and western NSW).
- When writing in Markdown cells, start each sentence is on a **new line**.
This makes it easy to see changes through git commits.
- Use Australian English in markdown cells and code comments.
- Use the [PEP8 standard](https://www.python.org/dev/peps/pep-0008/) for code. To make sure all code in the notebook is consistent, you can use the `jupyterlab_code_formatter` tool: select each code cell, then click `Edit` and then one of the `Apply X Formatter` options (`YAPF` or `Black` are recommended). This will reformat the code in the cell to a consistent style.
- In the final notebook cell, include a set of relevant tags which are used to build the DEA User Guide's [Tag Index](https://docs.dea.ga.gov.au/genindex.html). 
Use all lower-case, seperate words with spaces, and where possible re-use existing tags.
Ensure the tags cell below is in `Raw` format, rather than `Markdown` or `Code`.


### Overview
Digital Earth Australia (DEA) is a platform designed to:
* Catalogue large amounts of Earth Observation data
* Provide a Python based API for high performance querying and data access
* Give scientists and other users easy ability to perform exploratory data analysis
* Allow scalable continent scale processing of the stored data
* Track the provenance of all the contained data to allow for quality control and updates

For more information on the development of the DEA platform, please see [Dhu et al. 2017](https://doi.org/10.1080/20964471.2017.1402490).

### Description
This introduction to the DEA will review the dominant types of datasets catalogued in the platform.

Topics include
* a review of the satellite sensors whose data contributes to the DEA
* a review of the data format including:
  * band naming conventions
  * the coordinate reference scheme
  * data formatting with xarray


### Prerequisites
Users of this notebook should have a basic understanding of the use and format of the Jupyter Notebook.
To review these basics, see [Introduction_to_Jupyter](https://github.com/GeoscienceAustralia/dea-notebooks/tree/develop/Beginners_guide)

## Getting started
Digital Earth Australia catalogues data from a range of satellite sensors. 
The first DEA aquisitions of optical satellite imagery date from 1986 and include data from:
* Landsat 5, operational between March 1984 and January 2013
* Landsat 7, operational since April 1999
* Landsat 8, operational since February 2013
* Sentinel 2A, operational since June 2015
* Sentinel 2B, operational since March 2017

The Landsat missions are jointly operated by the United States Geological Survey (USGS) and National Aeronautics and Space Administration (NASA).
The Sentinel missions are operated by the European Space Agency (ESA).

The datasets generated by each of these sensors (satellites) are subtly different (Insert figure - Landsat and Sentinel sensor comparison).
For each sampled region of the electronmagnetic spectrum (EMS), the frequency range (band) differs between sensors. 
Similarly, the number of bands that are detected on each sensor also differs and their naming conventions are discussed in more detail below.

The spatial resolution also varies between the Landsat and Sentinel programs.
Landsat pixel sizes represent 30 m^2 of the land surface while Sentinel pixel sizes represent 10 m^2.

### Testing cell - edit figure/s and insert into 'Getting started' section with references
https://directory.eoportal.org/web/eoportal/satellite-missions/l/landsat-9 See figure 3/4 down page. Landsat 5 7 8 band comparison

https://www.usgs.gov/media/images/comparison-landsat-7-and-8-bands-sentinel-2 See figure comparing Sentinel 2 bands with Landsat 7 and 8

## Data format
### Band naming conventions
* Coastal aerosol
* Blue
* Green
* Red
* Nir (Near infra-red)
* Swir1 (Short wave infra-red 1)
* Swir2 (Short wave infra-red 2)

### Collection 3 product names
* ga_ls5t_ard_3 (Landsat 5 ARD)
* ga_ls7e_ard_3 (Landsat 7 ARD)
* ga_ls8c_ard_3 (Landsat 8 ARD)
* Sentinel 2???

### Output CRS
* EPSG:3577
* Discuss changes to path/row etc for scene sampling???

### Xarray
* run a test query to show a sample xarray. Explain how it constructed and what information is stored where/how

### Load packages
Use standard import commands; some are shown below. 
Begin with any `iPython` magic commands, followed by standard Python packages, then any additional functionality you need from the `Scripts` directory.

In [1]:
%matplotlib inline

import datacube
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys
import xarray as xr

sys.path.append("../Scripts")

In [2]:
# Testing cell

import datacube

# Temporary solution to connecting to datacube to account for 
# Collection 3 data being in a different database on the NCI
try:
    dc = datacube.Datacube(app='Collection_3', env='c3-samples')
except:
    dc = datacube.Datacube(app='Collection_3')

# Set up analysis data query. Compared to Collection 2, we need to 
# specify an 'output_crs' and 'resolution' (like Sentinel 2)
query = {'x': (142.20, 142.24),
         'y': (-32.44, -32.48),
         'time': ('1989-01-01', '1989-02-01'),
         'measurements': ['nbart_blue', 'fmask'],        
         'output_crs': 'EPSG:3577',
         'resolution': (-30, 30)}

# Load data
data = dc.load(product = 'ga_ls5t_ard_3', 
               group_by = 'solar_day', 
               **query)

print(data)

<xarray.Dataset>
Dimensions:     (time: 3, x: 137, y: 159)
Coordinates:
  * time        (time) datetime64[ns] 1989-01-13T23:50:33.098754 ... 1989-01-29T23:50:25.082071
  * y           (y) float64 -3.575e+06 -3.575e+06 ... -3.579e+06 -3.579e+06
  * x           (x) float64 9.5e+05 9.5e+05 9.5e+05 ... 9.54e+05 9.54e+05
Data variables:
    nbart_blue  (time, y, x) int16 1257 1242 1273 1288 ... 1322 1306 1273 1290
    fmask       (time, y, x) uint8 5 5 5 5 5 5 5 5 5 5 5 ... 5 5 5 5 5 5 5 5 5 5
Attributes:
    crs:      EPSG:3577


### Connect to the datacube
Give your datacube app a unique name that is consistent with the purpose of the notebook.

In [2]:
dc = datacube.Datacube(app="DEA_notebooks_template")

### Analysis parameters

An *optional* section to inform the user of any parameters they'll need to configure to run the notebook:
* `param_name_1`: Simple description (e.g. `example_value`). Advice about appropriate values to choose for this parameter.
* `param_name_2`: Simple description (e.g. `example_value`). Advice about appropriate values to choose for this parameter.


In [3]:
param_name_1 = "example_value"
param_name_2 = "example_value"

## Heading 1
Use headings to break up key steps/stages of the notebook.

Use markdown text for detailed, descriptive text explaining what the code below does and why it is needed.

In [4]:
# Use code comments for low-level documentation of code
a = 1

### Subheading 1
Use subheadings to break up steps within a single section.

In [5]:
# Use code comments for low-level documentation of code
b = 2

## Heading 2
Use markdown text for detailed, descriptive text explaining what the code below does and why it is needed.

In [6]:
# Use code comments for low-level documentation of code
c = 3

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Australia data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/GeoscienceAustralia/dea-notebooks).

**Last modified:** September 2019

**Compatible `datacube` version:** 

In [7]:
print(datacube.__version__)

1.7+43.gc873f3ea


## Tags
Browse all available tags on the DEA User Guide's [Tags Index](https://docs.dea.ga.gov.au/genindex.html)